
# Textgenerierung: Verwendung verschiedener Dekodierungsmethoden für die Spracherzeugung mit Transformatoren

[Credits to: https://github.com/huggingface/blog/blob/main/notebooks/02_how_to_generate.ipynb]

### **Einführung**

In den letzten Jahren ist das Interesse an der Generierung von Sprache dank des Aufkommens großer transformatorbasierter Sprachmodelle, die auf Millionen von Webseiten trainiert wurden, wie z. B. das berühmte [GPT2-Modell](https://openai.com/blog/better-language-models/) von OpenAI, gestiegen. Die Ergebnisse der konditionierten, offenen Spracherzeugung sind beeindruckend, z. B. [GPT2 on unicorns](https://openai.com/blog/better-language-models/#samples), [XLNet](https://medium.com/@amanrusia/xlnet-speaks-comparison-to-gpt-2-ea1a4e9ba39e), [Controlled language with CTRL](https://blog.einstein.ai/introducing-a-conditional-transformer-language-model-for-controllable-generation/). Neben der verbesserten Transformator-Architektur und der großen Menge an unüberwachten Trainingsdaten haben auch **bessere Dekodierungsmethoden** eine wichtige Rolle gespielt.

Dieses Notebook gibt einen kurzen Überblick über verschiedene Dekodierungsstrategien und zeigt vor allem, wie *Sie* diese mit sehr geringem Aufwand mit der beliebten Bibliothek `transformers` umsetzen können!

Alle der folgenden Funktionen können für die **autoregressive** Spracherzeugung verwendet werden ([hier](http://jalammar.github.io/illustrated-gpt2/) eine Auffrischung). Kurz gesagt, *autoregressive* Sprachgenerierung basiert auf der Annahme, dass die Wahrscheinlichkeitsverteilung einer Wortfolge in das Produkt der bedingten nächsten Wortverteilungen zerlegt werden kann:
$$ P(w_{1:T} | W_0 ) = \prod_{t=1}^T P(w_{t} | w_{1: t-1}, W_0) \text{ ,with } w_{1: 0} = \emptyset, $$

und $W_0$ ist die anfängliche *Kontext*-Wortfolge. Die Länge $T$ der Wortfolge wird üblicherweise *on-the-fly* bestimmt und entspricht dem Zeitschritt $t=T$, in dem das EOS-Token aus $P(w_{t} | w_{1: t-1}, W_{0})$ erzeugt wird.


Auto-regressive Spracherzeugung ist nun verfügbar für `GPT2`, `XLNet`, `OpenAi-GPT`, `CTRL`, `TransfoXL`, `XLM`, `Bart`, `T5` sowohl in PyTorch als auch in Tensorflow >= 2.0!

Wir geben einen Überblick über die derzeit bekanntesten Dekodierungsmethoden, vor allem *Greedy search*, *Beam search*, *Top-K sampling* und *Top-p sampling*.


Zunächst müssen Sie die Bibliothek "transformers" und das Modell laden. Wir werden GPT2 in Tensorflow 2.1 zur Demonstration verwenden, aber die API ist 1 zu 1 die gleiche für PyTorch.

In [1]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# das EOS-Token als PAD-Token hinzufügen, um Warnungen zu vermeiden
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


### **Greedy Search**

Die Greedy Search wählt einfach das Wort mit der höchsten Wahrscheinlichkeit als nächstes Wort: $w_t = argmax_{w}P(w | w_{1:t-1})$ in jedem Zeitschritt $t$. Die folgende Skizze zeigt die Greedy Search.

![Gierige Suche](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/greedy_search.png)

Ausgehend von dem Wort $\text{"The"}$, wählt der Greedy Algorithmus das nächste Wort mit der höchsten Wahrscheinlichkeit $\text{"nice"}$ und so weiter, so dass die endgültige generierte Wortfolge $\text{"The", "nice", "woman"}$ mit einer Gesamtwahrscheinlichkeit von $0,5 \mal 0,4 = 0,2$ ist.

Im Folgenden werden wir Wortfolgen mit GPT2 auf dem Kontext $(\text{"I", "enjoy", "walking", "with", "my", "cute", "dog"})$ erzeugen. Schauen wir uns an, wie die gierige Suche in `Transformers` wie folgt verwendet werden kann:

In [2]:
# Kodierung des Kontexts, der für die Generierung erforderlich ist
input_ids = tokenizer.encode('I enjoy walking with my cute dog', return_tensors='tf')

# Text generieren, bis die Ausgabelänge (die die Kontextlänge einschließt) 50 erreicht
greedy_output = model.generate(input_ids, max_length=50)

print("Ausgabe:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Ausgabe:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not sure if I'll


Na also! Wir haben unseren ersten kurzen Text mit GPT2 generiert 😊. Die generierten Wörter, die dem Kontext folgen, sind vernünftig, aber das Modell fängt schnell an, sich zu wiederholen! Dies ist ein sehr häufiges Problem bei der Spracherzeugung im Allgemeinen und scheint bei der Greedy- und Beam-Suche noch häufiger aufzutreten - siehe [Vijayakumar et al., 2016](https://arxiv.org/abs/1610.02424) und [Shao et al., 2017](https://arxiv.org/abs/1701.03185).

Der größte Nachteil der Greedy Search ist jedoch, dass sie Wörter mit hoher Wahrscheinlichkeit übersieht, die sich hinter einem Wort mit niedriger Wahrscheinlichkeit verbergen, wie in unserer Skizze oben zu sehen ist:

Das Wort $\text{"has"}$ mit seiner hohen bedingten Wahrscheinlichkeit von $0,9$ ist hinter dem Wort $\text{"dog"}$ versteckt, das nur die zweithöchste bedingte Wahrscheinlichkeit hat, so dass die gierige Suche die Wortfolge $\text{"The"}, \text{"dog"}, \text{"has"}$ übersieht.

Zum Glück gibt es die Strahlensuche, um dieses Problem zu lindern!


### **Beam Search**

Die Beam Search verringert das Risiko, versteckte Wortfolgen mit hoher Wahrscheinlichkeit zu übersehen, indem bei jedem Zeitschritt die wahrscheinlichsten `num_beams` der Hypothesen beibehalten und schließlich die Hypothese mit der insgesamt höchsten Wahrscheinlichkeit ausgewählt wird. Zur Veranschaulichung nehmen wir `num_beams=2`:

![Beam search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/beam_search.png)

Im Zeitschritt $1$ wird neben der wahrscheinlichsten Hypothese $\text{"The", "nice"}$ auch die zweitwahrscheinlichste $\text{"The", "dog"}$ verfolgt. Im Zeitschritt $2$ stellt die Strahlensuche fest, dass die Wortfolge $\text{"The", "dog", "has"}$ mit $0,36$ eine höhere Wahrscheinlichkeit hat als $\text{"The", "nice", "woman"}$, die $0,2$ hat. Toll, es hat die wahrscheinlichste Wortfolge in unserem Spielzeugbeispiel gefunden!

Die Balkensuche findet immer eine Ausgabesequenz mit höherer Wahrscheinlichkeit als die gierige Suche, aber es ist nicht garantiert, dass sie die wahrscheinlichste Ausgabe findet.

Schauen wir uns an, wie die Strahlensuche in `Transformers` verwendet werden kann. Wir setzen `num_beams > 1` und `early_stopping=True`, so dass die Erzeugung beendet ist, wenn alle Strahlhypothesen das EOS-Token erreicht haben.

In [3]:
# Beam Search und Frühstopp aktivieren
beam_output = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    early_stopping=True
)

print("Ausgabe:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Ausgabe:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I'm not sure if I'll ever be able to walk with him again. I'm not sure if I'll


Das Ergebnis ist zwar deutlich flüssiger, aber die Ausgabe enthält immer noch Wiederholungen der gleichen Wortfolgen.  
Eine einfache Abhilfe ist die Einführung von *n-Grammen* (*a.k.a.* Wortfolgen aus $n$ Wörtern), wie sie von [Paulus et al. (2017)](https://arxiv.org/abs/1705.04304) und [Klein et al. (2017)](https://arxiv.org/abs/1701.02810) eingeführt wurden. Die gängigste *n-Gramme*-Penalty stellt sicher, dass kein *n-Gramm* zweimal vorkommt, indem sie die Wahrscheinlichkeit der nächsten Wörter, die ein bereits gesehenes *n-Gramm* erzeugen könnten, manuell auf $0$ setzt.

Probieren wir es aus, indem wir `no_repeat_ngram_size=2` setzen, so dass kein *2-Gramm* zweimal erscheint:

In [4]:
# no_repeat_ngram_size auf 2 setzen
beam_output = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True
)

print("Ausgabe:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Ausgabe:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to take a break


Schön, das sieht viel besser aus! Wir können sehen, dass die Wiederholung nicht mehr vorkommt. Dennoch müssen *n-gram* penalties mit Vorsicht verwendet werden. Ein Artikel, der über die Stadt *New York* erstellt wird, sollte keine *2-Gramm*-Strafe verwenden, da sonst der Name der Stadt nur einmal im gesamten Text vorkommt!

Ein weiteres wichtiges Merkmal der Beam Search (Strahlensuche) ist, dass wir die Top-Strahlen nach der Generierung vergleichen und den generierten Strahl auswählen können, der am besten zu unserem Zweck passt.

In `transformers` setzen wir einfach den Parameter `num_return_sequences` auf die Anzahl der am höchsten bewerteten Balken, die zurückgegeben werden sollen. Stellen Sie aber sicher, dass `num_return_sequences <= num_beams`!

In [5]:
# set return_num_sequences > 1
beam_outputs = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    early_stopping=True
)

# jetzt haben wir 3 Ausgabesequenzen
print("Ausgabe:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Ausgabe:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to take a break
1: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to get back to
2: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with her again.

I've been thinking about this for a while now, and I think it's time for me to take a break
3: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with her again.

I've been thinking about this for a while now, and I think it's time for me to get back to
4: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about

Wie man sieht, unterscheiden sich die fünf Strahlenhypothesen nur geringfügig voneinander - was bei nur 5 Strahlen nicht allzu überraschend sein sollte.

Bei der offenen Generierung wurden in letzter Zeit einige Gründe angeführt, warum die Beam Search möglicherweise nicht die bestmögliche Option ist:

- Die Beam Search kann sehr gut bei Aufgaben funktionieren, bei denen die Länge der gewünschten Generierung mehr oder weniger vorhersehbar ist, wie bei der maschinellen Übersetzung oder Zusammenfassung - siehe [Murray et al. (2018)](https://arxiv.org/abs/1808.10006) und [Yang et al. (2018)](https://arxiv.org/abs/1808.09582). Dies gilt jedoch nicht für die Generierung mit offenem Ende, bei der die gewünschte Ausgabelänge stark variieren kann, z. B. bei der Generierung von Dialogen und Geschichten.

- Wir haben gesehen, dass die Beam Search stark unter der repetitiven Generierung leidet. Dies ist besonders schwer mit *n-Gramm*- oder anderen penalties bei der Generierung von Geschichten in den Griff zu bekommen, da die Suche nach einem guten Kompromiss zwischen erzwungener "Nicht-Wiederholung" und sich wiederholenden Zyklen identischer *n-Gramme* eine Menge Feinabstimmung erfordert.

- Wie in [Ari Holtzman et al. (2019)] (https://arxiv.org/abs/1904.09751) argumentiert wird, folgt hochwertige menschliche Sprache nicht einer Verteilung von mit hoher Wahrscheinlichkeit folgenden Wörtern. Mit anderen Worten: Als Menschen wollen wir, dass der generierte Text uns überrascht und nicht langweilig/vorhersehbar ist. Die Autoren zeigen dies sehr schön, indem sie die Wahrscheinlichkeit, die ein Modell einem menschlichen Text geben würde, im Vergleich zu dem, was die Beam Search tut, grafisch darstellen.

![alt text](https://blog.fastforwardlabs.com/images/2019/05/Screen_Shot_2019_05_08_at_3_06_36_PM-1557342561886.png)


Also lasst uns aufhören, langweilig zu sein, und etwas Zufälligkeit einführen 🤪.

### **Sampling**

In seiner einfachsten Form bedeutet Sampling die zufällige Auswahl des nächsten Wortes $w_t$ gemäß seiner bedingten Wahrscheinlichkeitsverteilung:

$$w_t \sim P(w|w_{1:t-1})$$

Anhand des obigen Beispiels veranschaulicht die folgende Grafik die Spracherzeugung beim Sampling.

![vanilla_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/sampling_search.png)

Es wird deutlich, dass die Spracherzeugung mittels Sampling nicht mehr *deterministisch* ist. Das Wort
$\text{"car"}$ wird aus der bedingten Wahrscheinlichkeitsverteilung $P(w | \text{"The"})$ entnommen, gefolgt von einer Stichprobe $\text{"drives"}$ aus $P(w | \text{"The"}, \text{"car"})$.

In `transformers` setzen wir `do_sample=True` und deaktivieren *Top-K* Sampling (mehr dazu später) über `top_k=0`. Im Folgenden werden wir zur Veranschaulichung `random_seed=0` festlegen. Es steht Ihnen frei, die `random_seed` zu ändern, um mit dem Modell herumzuspielen.


In [6]:
# Seed setzen, um Ergebnisse zu reproduzieren. Sie können den Seed aber auch ändern, um andere Ergebnisse zu erhalten
tf.random.set_seed(0)

# Stichproben aktivieren und top_k deaktivieren, indem top_k sampling auf 0 gesetzt wird
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=0
)

print("Ausgabe:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Ausgabe:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog and never seem to get nervous until it turns into battery unfrosted. (Laughs) Go ahead with the cover and bring it on. "3 Light Doppelgangers" by AFOB

…



Interessant! Der Text scheint in Ordnung zu sein - aber wenn man genauer hinsieht, ist er nicht sehr kohärent. Die *3-Gramme* *New Hand Sense* und *Local Batte Harness* sind sehr seltsam und klingen nicht so, als wären sie von einem Menschen geschrieben worden. Das ist das große Problem beim Sampling von Wortfolgen: Die Modelle erzeugen oft inkohärentes Kauderwelsch, *cf.* [Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751).

Ein Trick besteht darin, die Verteilung $P(w|w_{1:t-1})$ schärfer zu machen (die Wahrscheinlichkeit von Wörtern mit hoher Wahrscheinlichkeit zu erhöhen und die Wahrscheinlichkeit von Wörtern mit niedriger Wahrscheinlichkeit zu verringern), indem man die so genannte "Temperatur" des [softmax](https://en.wikipedia.org/wiki/Softmax_function#Smooth_arg_max) senkt.

Ein Beispiel für die Anwendung der Temperatur auf unser obiges Beispiel könnte folgendermaßen aussehen.

![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/sampling_search_with_temp.png?raw=true)

Die bedingte Verteilung des nächsten Wortes in Schritt $t=1$ wird viel schärfer, so dass das Wort $\text{"car"}$ fast keine Chance mehr hat, ausgewählt zu werden.


Schauen wir uns an, wie wir die Verteilung in der Bibliothek abkühlen können, indem wir `temperature=0.7` einstellen:

OK. Es gibt weniger seltsame n-Gramme und die Ausgabe ist jetzt etwas kohärenter! Während die Anwendung von Temperatur eine Verteilung weniger zufällig machen kann, wird das temperaturskalierte Sampling im Grenzfall, wenn `Temperatur` auf $0$ gesetzt wird, gleichbedeutend mit gieriger Dekodierung und leidet unter den gleichen Problemen wie zuvor.



In [18]:
# Seed setzen, um Ergebnisse zu reproduzieren. Es steht Ihnen frei, den Seed zu ändern, um andere Ergebnisse zu erhalten
tf.random.set_seed(0)

# Temperatur verwenden, um die Empfindlichkeit gegenüber Kandidaten mit geringer Wahrscheinlichkeit zu verringern
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=50, 
    top_k=0, 
    temperature=0.7
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog. I've always been an avid dog lover. My favorite type of dog is one that has been trained to do many things. I do not have any special preferences for particular breeds. I don't think I have


### **Top-K Sampling**

[Fan et. al (2018)](https://arxiv.org/pdf/1805.04833.pdf) führte ein einfaches, aber sehr leistungsfähiges Stichprobenverfahren ein, das sogenannte ***Top-K***-Sampling. Beim *Top-K*-Sampling werden die *K* wahrscheinlichsten nächsten Wörter gefiltert und die Wahrscheinlichkeitsmasse wird nur auf diese *K* nächsten Wörter umverteilt.
GPT2 hat dieses Stichprobenverfahren übernommen, was einer der Gründe für seinen Erfolg bei der Generierung von Geschichten war.

Zur besseren Veranschaulichung des *Top-K*-Samplings erweitern wir im obigen Beispiel den Bereich der für beide Sampling-Schritte verwendeten Wörter von 3 auf 10 Wörter.

![top_k_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/top_k_sampling.png)

Nachdem wir $K = 6$ gesetzt haben, beschränken wir unseren Stichprobenpool in beiden Schritten auf 6 Wörter. Während die 6 wahrscheinlichsten Wörter, definiert als $V_{\text{top-K}}$, im ersten Schritt nur *ca.* zwei Drittel der gesamten Wahrscheinlichkeitsmasse umfassen, schließen sie im zweiten Schritt fast die gesamte Wahrscheinlichkeitsmasse ein. Nichtsdestotrotz sehen wir, dass sie erfolgreich die eher seltsamen Kandidaten $\text{"nicht", "die", "klein", "gesagt"}$ eliminiert
im zweiten Schritt der Stichprobenziehung.


Schauen wir uns an, wie *Top-K* in der Bibliothek verwendet werden kann, indem wir `top_k=50` einstellen:

In [19]:
# Seed setzen, um Ergebnisse zu reproduzieren. Es steht Ihnen frei, den Seed zu ändern, um andere Ergebnisse zu erhalten
tf.random.set_seed(0)

# top_k auf 50 setzen
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=50
)

print("Ausgabe:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Ausgabe:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog in the street from 2 1/2 miles (just before sunset) to 3 miles. But there are a few things that you need to know! It's NOT A REPUBLICANS EXCELLENT!


Gar nicht so schlecht! Der Text ist wohl der bisher am *menschlichsten* klingende Text.
Ein Problem beim *Top-K*-Sampling ist jedoch, dass es die Anzahl der Wörter, die aus der nächsten Wortwahrscheinlichkeitsverteilung $P(w|w_{1:t-1})$ gefiltert werden, nicht dynamisch anpasst.
Dies kann problematisch sein, da einige Wörter aus einer sehr scharfen Verteilung (Verteilung rechts im obigen Diagramm), andere hingegen aus einer viel flacheren Verteilung (Verteilung links im obigen Diagramm) gezogen werden könnten.

In Schritt $t=1$ eliminiert *Top-K* die Möglichkeit
$\text{"people", "big", "house", "cat"}$, die als vernünftige Kandidaten erscheinen. Andererseits nimmt das Verfahren in Schritt $t=2$ die wohl schlecht passenden Wörter $\text{"down", "a"}$ in den Stichprobenpool auf. Eine Begrenzung des Stichprobenpools auf eine feste Größe *K* könnte also dazu führen, dass das Modell bei scharfen Verteilungen Kauderwelsch produziert und bei flachen Verteilungen die Kreativität des Modells einschränkt.
Diese Intuition hat [Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751) dazu veranlasst, das ***Top-p***- oder ***nucleus***-Sampling zu entwickeln.



### **Top-p (nucleus) sampling**

Anstatt nur aus den wahrscheinlichsten *K* Wörtern eine Stichprobe zu ziehen, wird bei *Top-p* aus der kleinstmöglichen Menge von Wörtern ausgewählt, deren kumulative Wahrscheinlichkeit die Wahrscheinlichkeit *p* übersteigt. Die Wahrscheinlichkeitsmasse wird dann auf diese Menge von Wörtern umverteilt. Auf diese Weise kann die Größe der Wortmenge (*a.k.a.* die Anzahl der Wörter in der Menge) entsprechend der Wahrscheinlichkeitsverteilung des nächsten Wortes dynamisch zu- und abnehmen. Ok, das war sehr wortreich, lassen Sie uns das veranschaulichen.

![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/top_p_sampling.png?raw=true)

Nachdem $p=0,92$ festgelegt wurde, wählt *Top-p* die *minimale* Anzahl von Wörtern aus, die zusammen $p=92\%$ der Wahrscheinlichkeitsmasse überschreiten, definiert als $V_{\text{top-p}}$. Im ersten Beispiel umfasste dies die 9 wahrscheinlichsten Wörter, während im zweiten Beispiel nur die obersten 3 Wörter ausgewählt werden müssen, um 92 % zu überschreiten. Eigentlich ganz einfach! Es ist zu erkennen, dass eine große Anzahl von Wörtern erhalten bleibt, bei denen das nächste Wort wohl weniger vorhersehbar ist, *z.B.* $P(w | \text{"The"})$, und nur wenige Wörter, bei denen das nächste Wort vorhersehbarer erscheint, *z.B.* $P(w | \text{"The", "car"})$.

Also gut, dann probieren wir es mal mit `transformers` aus!
Wir aktivieren *Top-p* Sampling, indem wir `0 < top_p < 1` setzen:

In [20]:
# Seed setzen, um Ergebnisse zu reproduzieren. Es steht Ihnen frei, den Seed zu ändern, um andere Ergebnisse zu erhalten
tf.random.set_seed(0)

# Deaktivieren Sie das top_k-Sampling und wählen Sie nur die 92% wahrscheinlichsten Wörter aus.
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_p=0.92,
    top_k=0
)

print("Ausgabe:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Ausgabe:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog and very often see her immediately excited at moving on." - Alicia Davis

Recently, running around Los Angeles on an expeditions had me thinking, "What'd I get for free here?" I worked as a


Toll, das klingt, als hätte es ein Mensch geschrieben. Nun, vielleicht noch nicht ganz.

Obwohl *Top-p* in der Theorie eleganter erscheint als *Top-K*, funktionieren beide Methoden in der Praxis gut. *Top-p* kann auch in Kombination mit *Top-K* verwendet werden, wodurch sehr niedrig eingestufte Wörter vermieden werden können und gleichzeitig eine dynamische Auswahl möglich ist.

Um schließlich mehrere unabhängig voneinander abgetastete Ausgaben zu erhalten, können wir *wieder* den Parameter `num_return_sequences > 1` setzen:

In [21]:
# Seed setzen, um Ergebnisse zu reproduzieren. Es steht Ihnen frei, den Seed zu ändern, um andere Ergebnisse zu erhalten
tf.random.set_seed(0)

# top_k = 50 und top_p = 0,95 und num_return_sequences = 3 setzen
sample_outputs = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=50,
    top_p=0.95,
    num_return_sequences=3
)

print("Ausgabe:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Ausgabe:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog

In my life I love having fun

What does it cost to feed a cat to get a heart attack?

I think, this is actually a really good question: what are some of the
1: I enjoy walking with my cute dog."

We didn't realize it until the next day that her husband had become very busy, so we were not surprised when the dog went to visit him outside his apartment and suddenly she had the courage to ask
2: I enjoy walking with my cute dog, I like to get a dog from the grocery store and it's been a pleasure to sit and talk to her (when she is sleeping), and she's been so happy with our visit to the supermarket (we


Cool, jetzt solltest du alle Werkzeuge haben, um dein Modell deine Geschichten mit `Transformers` schreiben zu lassen!

### **Fazit**

Als *ad-hoc* Dekodierungsmethoden scheinen *top-p* und *top-K* Sampling einen flüssigeren Text zu produzieren als die traditionelle *Greedy*- und *Beam*-Suche bei der Erzeugung von Sprache mit offenem Ende.
In letzter Zeit mehren sich jedoch die Hinweise darauf, dass die offensichtlichen Schwächen von *Greedy* und *Beam*-Suche - vor allem die Erzeugung sich wiederholender Wortfolgen - durch das Modell (insbesondere die Art und Weise, wie das Modell trainiert wird) und nicht durch die Dekodierungsmethode verursacht werden, *vgl.* [Welleck et al. (2019)](https://arxiv.org/pdf/1908.04319.pdf). Wie in [Welleck et al. (2020)](https://arxiv.org/abs/2002.02492) gezeigt wird, scheinen auch *top-K* und *top-p* Sampling unter der Erzeugung sich wiederholender Wortfolgen zu leiden.

In [Welleck et al. (2019)] (https://arxiv.org/pdf/1908.04319.pdf) zeigen die Autoren, dass die *Beam*-Suche nach menschlicher Einschätzung flüssigere Texte erzeugen kann als das *Top-p*-Sampling, wenn man das Trainingsziel des Modells anpasst.

Die Generierung offener Sprache ist ein sich rasch entwickelndes Forschungsgebiet, und wie so oft gibt es auch hier keine Einheitsmethode, so dass man sehen muss, was im eigenen Anwendungsfall am besten funktioniert.

Gut, dass *du* in `transfomers` alle verschiedenen Dekodierungsmethoden ausprobieren kannst 🤗.

Dies war eine kurze Einführung in die Verwendung verschiedener Dekodierungsmethoden in "Transformatoren" und in die jüngsten Trends bei der Erzeugung von Sprachen mit offenem Ende.

Feedback und Fragen sind auf dem [Github-Repository] (https://github.com/huggingface/transformers) sehr willkommen.

Weitere lustige Geschichten finden Sie unter [Schreiben mit Transformers] (https://transformer.huggingface.co).

Vielen Dank an alle, die zu diesem Blogbeitrag beigetragen haben: Alexander Rush, Julien Chaumand, Thomas Wolf, Victor Sanh, Sam Shleifer, Clément Delangue, Yacine Jernite, Oliver Åstrand und John de Wasseige.


### **Anhang**

Es gibt eine Reihe zusätzlicher Parameter für die Methode `generate`, die oben nicht erwähnt wurden. Wir werden sie hier kurz erklären!

- min_length" kann verwendet werden, um das Modell zu zwingen, kein EOS-Token zu produzieren (= den Satz nicht zu beenden), bevor "min_length" erreicht ist. Dies wird häufig bei Zusammenfassungen verwendet, kann aber auch allgemein nützlich sein, wenn der Benutzer längere Ausgaben haben möchte.
- repetition_penalty" kann verwendet werden, um Wörter zu bestrafen, die bereits generiert wurden oder zum Kontext gehören. Sie wurde erstmals von [Kesker et al. (2019)](https://arxiv.org/abs/1909.05858) eingeführt und wird auch im Trainingsziel in [Welleck et al. (2019)](https://arxiv.org/pdf/1908.04319.pdf) verwendet. Es kann recht effektiv sein, um Wiederholungen zu verhindern, scheint aber sehr empfindlich auf verschiedene Modelle und Anwendungsfälle zu reagieren, *siehe z.B.* diese [Diskussion](https://github.com/huggingface/transformers/pull/2303) auf Github.

- `attention_mask` kann verwendet werden, um aufgefüllte Token zu maskieren
- `pad_token_id`, `bos_token_id`, `eos_token_id`: Wenn das Modell diese Token nicht standardmäßig enthält, kann der Benutzer manuell andere Token-IDs auswählen, um sie zu repräsentieren.

Für weitere Informationen schauen Sie bitte auch in die Funktion `generate` [docstring](https://huggingface.co/transformers/main_classes/model.html?highlight=generate#transformers.TFPreTrainedModel.generate).

# Weitere Links (mit Colab Notebooks)

https://keras.io/examples/generative/gpt2_text_generation_with_kerasnlp/

https://keras.io/examples/generative/gpt2_text_generation_with_kerasnlp/